<h1>Read The data</h1>

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [24]:
concrete_data = pd.read_csv("concrete_data.csv")
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


## Minimize to data


In [25]:
import scipy.stats as stats
minimize_concrete_data=concrete_data.apply(stats.zscore)
concrete_data.shape
minimize_concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,2.477915,-0.856888,-0.847144,-0.916764,-0.620448,0.863154,-1.217670,-0.279733,2.645408
1,2.477915,-0.856888,-0.847144,-0.916764,-0.620448,1.056164,-1.217670,-0.279733,1.561421
2,0.491425,0.795526,-0.847144,2.175461,-1.039143,-0.526517,-2.240917,3.553066,0.266627
3,0.491425,0.795526,-0.847144,2.175461,-1.039143,-0.526517,-2.240917,5.057677,0.313340
4,-0.790459,0.678408,-0.847144,0.488793,-1.039143,0.070527,0.647884,4.978487,0.507979


In [26]:
concrete_data_columns = minimize_concrete_data.columns

predictors = minimize_concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = minimize_concrete_data['Strength'] # Strength column



In [27]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.30)
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors.head()



,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.477915,-0.856888,-0.847144,-0.916764,-0.620448,0.863154,-1.217670,-0.279733
1,2.477915,-0.856888,-0.847144,-0.916764,-0.620448,1.056164,-1.217670,-0.279733
2,0.491425,0.795526,-0.847144,2.175461,-1.039143,-0.526517,-2.240917,3.553066
3,0.491425,0.795526,-0.847144,2.175461,-1.039143,-0.526517,-2.240917,5.057677
4,-0.790459,0.678408,-0.847144,0.488793,-1.039143,0.070527,0.647884,4.978487


In [28]:
n_cols = predictors_norm.shape[1] # number of predictors
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


## Model Creating


In [29]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_squared_error


In [30]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [31]:
model = regression_model()

In [32]:
mean_error=[]
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.30)
    predictors_norm = (predictors - predictors.mean()) / predictors.std()
    predictors_norm.head()
    n_cols = predictors_norm.shape[1] # number of predictors
    model.fit(X_train, y_train, validation_split=0.2, epochs=50, verbose=2)
    pred_y=model.predict(X_test)
    mean_error.append(mean_squared_error(y_test, pred_y))
    print(i)
    
    
    



Train on 576 samples, validate on 145 samples
Epoch 1/50


2023-10-15 12:27:49.245251: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2023-10-15 12:27:49.265598: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394290000 Hz
2023-10-15 12:27:49.266498: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x557e6d813b10 executing computations on platform Host. Devices:
2023-10-15 12:27:49.266576: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2023-10-15 12:27:49.495342: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

 - 1s - loss: 0.9560 - val_loss: 0.8290
Epoch 2/50
 - 0s - loss: 0.8666 - val_loss: 0.7845
Epoch 3/50
 - 0s - loss: 0.8051 - val_loss: 0.7405
Epoch 4/50
 - 0s - loss: 0.7538 - val_loss: 0.7003
Epoch 5/50
 - 0s - loss: 0.7066 - val_loss: 0.6662
Epoch 6/50
 - 0s - loss: 0.6614 - val_loss: 0.6295
Epoch 7/50
 - 0s - loss: 0.6180 - val_loss: 0.5989
Epoch 8/50
 - 0s - loss: 0.5805 - val_loss: 0.5679
Epoch 9/50
 - 0s - loss: 0.5448 - val_loss: 0.5403
Epoch 10/50
 - 0s - loss: 0.5152 - val_loss: 0.5144
Epoch 11/50
 - 0s - loss: 0.4912 - val_loss: 0.4935
Epoch 12/50
 - 0s - loss: 0.4702 - val_loss: 0.4717
Epoch 13/50
 - 0s - loss: 0.4535 - val_loss: 0.4512
Epoch 14/50
 - 0s - loss: 0.4386 - val_loss: 0.4315
Epoch 15/50
 - 0s - loss: 0.4244 - val_loss: 0.4195
Epoch 16/50
 - 0s - loss: 0.4096 - val_loss: 0.3981
Epoch 17/50
 - 0s - loss: 0.3956 - val_loss: 0.3848
Epoch 18/50
 - 0s - loss: 0.3827 - val_loss: 0.3732
Epoch 19/50
 - 0s - loss: 0.3703 - val_loss: 0.3550
Epoch 20/50
 - 0s - loss: 0.3546

In [33]:
# Mean and standart devination

def calc_mean(data):
    data_lenght=len(data)
    if data_lenght <= 1:
        return data
    else:
        return sum(data) / data_lenght
    
def calc_std(data):
    sd = 0.0 # std
    data_lenght = len(data)
    if data_lenght <= 1:
        return 0.0
    else:
        for _ in range(data_lenght):
            sd += (float(data[_]) - calc_mean(data)) ** 2
        sd = (sd / float(data_lenght)) ** 0.5
        return sd
    

In [34]:
mean=calc_mean(mean_error)
std=calc_std(mean_error)

In [35]:
print(mean)
print(std)

0.09047256606463693
0.028108386833765667
